# Clustering Crypto

In [1]:
!pip install -U altair

     |████████████████████████████████| 727 kB 19.7 MB/s eta 0:00:01


In [35]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import altair as alt

### Fetching Cryptocurrency Data

In [36]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [37]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
response_data = requests.get(url)

# Get content
response_content = response_data.content

import json

# Formatting as json
data = response_data.json()

# Add indents to JSON and output to screen
# print(json.dumps(data, indent=4))

# Create a Dataframe
df = pd.DataFrame(data['Data'])

# Transpose DF
crypto_df = df.T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi,IsUsedInNft
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [39]:
crypto_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'BlockNumber', 'NetHashesPerSecond',
       'BlockReward', 'BlockTime', 'AssetLaunchDate', 'MaxSupply',
       'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress',
       'DecimalPoints', 'Difficulty', 'IsUsedInDefi', 'IsUsedInNft'],
      dtype='object')

In [40]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
c_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
c_df['MaxSupply'].count()
c_df.shape

(6303, 6)

In [41]:
# Keep only cryptocurrencies that are trading
cr_df = c_df[c_df.IsTrading == True]
cr_df.shape

(5024, 6)

In [42]:
# Keep only cryptocurrencies with a working algorithm
cry_df = cr_df[cr_df.Algorithm != 'N/A']
cry_df.head(25)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
1337,EliteCoin,X13,True,PoW/PoS,0,0
2015,2015 coin,X11,True,PoW/PoS,0,0
BTCD,BitcoinDark,SHA-256,True,PoW/PoS,NaN,NaN
CRAIG,CraigsCoin,X11,True,PoS,NaN,NaN
XBS,Bitstake,X11,True,PoW/PoS,NaN,NaN


In [43]:
# Remove the "IsTrading" column
cry_df.drop(['IsTrading'], axis=1, inplace=True)
cry_df.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(1487, 5)

In [44]:
# Remove rows with at least 1 null value
cry_df.dropna(inplace=True)
cry_df.shape

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(279, 5)

In [45]:
# Remove rows with cryptocurrencies having no coins mined
cryp_df = cry_df[cry_df.TotalCoinsMined > 0]
cryp_df.shape

(136, 5)

In [46]:
# Remove rows with 'N/A' values
crypt_df = cryp_df[cryp_df != 'N/A']
crypt_df.shape

(136, 5)

In [47]:
# Remove rows with at least 1 null value - AGAIN SINCE NULL VALUES RE_EMERGE
crypt_df.dropna(inplace=True)
crypt_df.shape

(118, 5)

In [48]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name_df = crypt_df['CoinName']
coins_name_df.head()

NVC           NovaCoin
XCP       CounterParty
NSR           NuShares
MONA          MonaCoin
TRI     Triangles Coin
Name: CoinName, dtype: object

In [49]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypt_df.drop(['CoinName'], axis=1, inplace=True)

In [50]:
crypt_df.head(25)

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,Scrypt,PoW/PoS,3.19677e+06,-1
XCP,SHA-256,PoW,2.61508e+06,-1
NSR,PoS,PoS,5.85576e+09,0
MONA,Scrypt,PoW,8.1167e+07,-1
TRI,X13,PoW/PoS,175041,0
EMC,SHA-256,PoW/PoS,4.72098e+07,-1
SAFEX,Scrypt,PoC,2147483647,-1
CMTC,Scrypt,PoW,872830,0
XSN,X11,TPoS,1.18876e+08,-1
CHAT,Scrypt,PoW/PoS,1000000000,-1


In [51]:
# Create dummy variables for text features
X = pd.get_dummies(crypt_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_BEP-2,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,Algorithm_CryptoNight,Algorithm_CryptoNight-Heavy,...,ProofType_PoW,ProofType_PoW/PoS,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_Proof of Authority,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,ProofType_mPoW
NVC,3.19677e+06,-1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
XCP,2.61508e+06,-1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
NSR,5.85576e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MONA,8.1167e+07,-1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TRI,175041,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [52]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
X_scaled[:3]

array([[-0.17595775, -0.13641368, -0.09245003, -0.13130643, -0.09245003,
        -0.09245003, -0.13130643, -0.13130643, -0.13130643, -0.09245003,
        -0.09245003, -0.2511236 , -0.13130643, -0.09245003, -0.09245003,
        -0.09245003, -0.30429031, -0.09245003, -0.21035158, -0.09245003,
        -0.09245003, -0.13130643, -0.09245003, -0.09245003, -0.09245003,
        -0.09245003, -0.09245003, -0.16151457, -0.09245003, -0.09245003,
        -0.13130643, -0.16151457, -0.09245003, -0.16151457, -0.13130643,
        -0.33646329, -0.13130643, -0.09245003, -0.09245003, -0.09245003,
         2.3570226 , -0.09245003, -0.09245003, -0.09245003, -0.09245003,
        -0.09245003, -0.18731716, -0.09245003, -0.21035158, -0.09245003,
        -0.09245003, -0.09245003, -0.09245003, -0.26967994, -0.09245003,
        -0.09245003, -0.13130643, -0.13130643, -0.09245003, -0.28734789,
        -0.09245003, -0.09245003, -1.        ,  2.03234711, -0.09245003,
        -0.09245003, -0.09245003, -0.09245003, -0.0

### Reducing Dimensions Using PCA

In [53]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_scaled_pca = pca.fit_transform(X_scaled)
crypto_scaled_pca[:5]

array([[-1.23581912, -0.74227279, -0.33264759],
       [ 1.69382042, -0.15067879, -0.02463105],
       [-1.26921063, -0.01203079, -0.48430153],
       [ 0.74799384, -0.68285636, -0.21286348],
       [-1.97756725, -0.67439386, -0.38834845]])

In [54]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(crypto_scaled_pca, columns=['PC 1', 'PC 2', 'PC 3'])
pcs_df.index = X.index
pcs_df.head()

,PC 1,PC 2,PC 3
NVC,-1.235819,-0.742273,-0.332648
XCP,1.693820,-0.150679,-0.024631
NSR,-1.269211,-0.012031,-0.484302
MONA,0.747994,-0.682856,-0.212863
TRI,-1.977567,-0.674394,-0.388348


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [55]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using Altair
alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia')

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [56]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=5)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypt_df.join(pcs_df).join(coins_name_df)
clustered_df['class'] = model.labels_
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,CoinName,class
NVC,Scrypt,PoW/PoS,3.19677e+06,-1,-1.235819,-0.742273,-0.332648,NovaCoin,0
XCP,SHA-256,PoW,2.61508e+06,-1,1.693820,-0.150679,-0.024631,CounterParty,1
NSR,PoS,PoS,5.85576e+09,0,-1.269211,-0.012031,-0.484302,NuShares,0
MONA,Scrypt,PoW,8.1167e+07,-1,0.747994,-0.682856,-0.212863,MonaCoin,1
TRI,X13,PoW/PoS,175041,0,-1.977567,-0.674394,-0.388348,Triangles Coin,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [57]:
# Create a 3D-Scatter with the PCA data and the clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC 1',
    y='PC 2',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "MaxSupply"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [58]:
# Table with tradable cryptos
display_df = clustered_df.drop(columns=['PC 1', 'PC 2', 'PC 3'])
display(display_df)

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,CoinName,class
NVC,Scrypt,PoW/PoS,3.19677e+06,-1,NovaCoin,0
XCP,SHA-256,PoW,2.61508e+06,-1,CounterParty,1
NSR,PoS,PoS,5.85576e+09,0,NuShares,0
MONA,Scrypt,PoW,8.1167e+07,-1,MonaCoin,1
TRI,X13,PoW/PoS,175041,0,Triangles Coin,0
...,...,...,...,...,...,...
BTC,SHA-256,PoW,18670175,2.1e+07,Bitcoin,1
BNB,BEP-2,PoSA,1.70534e+08,1.70534e+08,Binance Coin,0
AION,"Equihash210,9",PoW/PoS,492427074,-1,Aion,0
OXYC,DPoS,DPoS,1.12238e+09,-1,Oxycoin,2


In [59]:
# Print the total number of tradable cryptocurrencies
print(f"The total number of tradable cryptocurrencies: {clustered_df.CoinName.count()}")

The total number of tradable cryptocurrencies: 118


#### Scatter Plot with Tradable Cryptocurrencies

In [60]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='TotalCoinsMined',
    y='MaxSupply',
    tooltip=["CoinName"]
).interactive()

alt.Chart(...)